In [331]:
import requests
import json
import numpy as np
from sklearn.model_selection import train_test_split
import time

In [332]:
WIT_API_HOST = 'https://api.wit.ai'
WIT_API_VERSION = '20200513'

In [351]:
with open('token.txt') as file:
    TOKEN = file.read()
    
HEADERS = {
    'authorization': 'Bearer ' + TOKEN,
    'accept': 'application/vnd.wit.' + WIT_API_VERSION + '+json'
}

In [334]:
def getQueries():
    rsp = requests.get(WIT_API_HOST + '/utterances', headers=HEADERS, params={'limit': 1000})
    return rsp.json()

In [335]:
def delQueries(queries):
    if not queries:
        print('empty list, aborting')
        return
    confirm = input('CONFIRM DELETION: ')
    if confirm != 'yes':
        print('aborting delete')
        return
    print('deleting the following. \n')
    print(queries)
    
    queriesDelete = [{k: v for k, v in query.items() if k.startswith('text')} for query in queries]
    rsp = requests.delete(WIT_API_HOST + '/utterances', headers=HEADERS, data=str(queriesDelete))
    
    print(rsp.json())

In [336]:
# [a,b,c]
def getExamples(filename):
    examples = np.loadtxt(filename, dtype='str', delimiter='\n')
    return examples

In [337]:
# [[a,x],[b,x],[c,x]]
def labelExamples(examples, label):
    reshaped = np.reshape(examples, (-1, 1))
    labels = np.full(examples.shape[0], label).reshape(-1, 1)
    return np.concatenate((reshaped, labels), 1)

In [338]:
# [ task1 [[a,x],[b,x],[c,x]] , task2 [[d,y],[e,y],[f,y]] ]
def getLabeledExamples(filenames, labels):
    data = []
    for i in range(0, len(filenames)):
        labeled = labelExamples(getExamples(filenames[i]), labels[i])
        data.append(labeled)
    return data

In [339]:
# [ train: task1 + task2 , test: task1 + task2 ]
def formTrainTest(data, split):
    trains = []
    tests = []
    for dataset in data:
        if (split == 'alt'):
            a = dataset[0::2]
            b = dataset[1::2]
        elif (split == 'full'):
            a = dataset
            b = []
        else:
            a, b = train_test_split(dataset, test_size=split)
        trains.append(a)
        tests.append(b)
        
    train = np.empty((0,2))
    test = np.empty((0,2))
    for i in range(len(trains)):
        train = np.concatenate((train, trains[i]))
        test = np.concatenate((test, tests[i]))
        
    return train, test

In [340]:
def examplesToJson(examples):
    json = []
    for example in examples:
        json.append({'text': example[0], 'intent': example[1], 'entities': [], 'traits': []})
    return json

In [346]:
data = getLabeledExamples(['class.txt', 'reg.txt'], ['task_class', 'task_reg'])
train, test = formTrainTest(data, 'alt')
trainJson = examplesToJson(train)
testJson = examplesToJson(test)

In [347]:
#str(trainJson)

In [352]:
rsp = requests.post(WIT_API_HOST + '/utterances', headers=HEADERS, data=str(trainJson))
rsp.json()

{'sent': True, 'n': 88}

In [326]:
time.sleep(60) # DELAY - ALLOW IT TO TRAIN

In [355]:
for testExample in testJson:
    rsp = requests.get(WIT_API_HOST + '/message', headers=HEADERS, params={'q': testExample['text']})
    rspJson = rsp.json()
    print(rspJson)
    
    print(len(rspJson['intents']) > 0 and rspJson['intents'][0]['name'] == testExample['intent'])
    print()

{'text': 'classification of audio samples', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.9993}], 'entities': {'subject:subject': [{'id': '3187784944605415', 'name': 'subject', 'role': 'subject', 'start': 18, 'end': 31, 'body': 'audio samples', 'confidence': 0.9128, 'entities': [], 'suggested': True, 'value': 'audio samples', 'type': 'value'}]}, 'traits': {}}
True

{'text': 'classifier to identify high-risk populations', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.997}], 'entities': {'subject:subject': [{'id': '3187784944605415', 'name': 'subject', 'role': 'subject', 'start': 11, 'end': 44, 'body': 'to identify high-risk populations', 'confidence': 0.7739, 'entities': [], 'suggested': True, 'value': 'to identify high-risk populations', 'type': 'value'}]}, 'traits': {}}
True

{'text': 'facial recognition', 'intents': [{'id': '1728532153951407', 'name': 'task_class', 'confidence': 0.887}], 'entities': {'subject:subject': [{'

In [345]:
len(getQueries())

1

In [320]:
delQueries(getQueries())

CONFIRM DELETION:  yes


deleting the following. 

[{'text': 'phishing detection', 'entities': [], 'traits': [], 'intent': {'id': '1728532153951407', 'name': 'task_class'}}, {'text': 'project the quantity of drinks', 'entities': [], 'traits': [], 'intent': {'id': '184248982902938', 'name': 'task_reg'}}, {'text': 'project median incomes for 2023', 'entities': [], 'traits': [], 'intent': {'id': '184248982902938', 'name': 'task_reg'}}, {'text': 'classifies spam texts', 'entities': [], 'traits': [], 'intent': {'id': '1728532153951407', 'name': 'task_class'}}, {'text': 'decide if an email was spoofed', 'entities': [], 'traits': [], 'intent': {'id': '1728532153951407', 'name': 'task_class'}}, {'text': 'forecast the weather', 'entities': [], 'traits': [], 'intent': {'id': '184248982902938', 'name': 'task_reg'}}, {'text': 'label photos by if they are authentic or not', 'entities': [], 'traits': [], 'intent': {'id': '1728532153951407', 'name': 'task_class'}}, {'text': 'detect fraud', 'entities': [], 'traits': [], 'inte